In [37]:
import pandas as pd
import numpy as np

In [38]:
df = pd.read_csv('./gz_recipe.csv')
df.head()

,Unnamed: 0,Nome,Categoria,Link,Persone/Pezzi,Ingredienti,Steps
0,0,Tiramisù,Dolci,https://ricette.giallozafferano.it/Tiramisu.html,8,"[['Mascarpone', '750g'], ['Uova', '260g'], ['S...",Per preparare il tiramisù preparate il caffé c...
1,1,Cookies,Dolci,https://ricette.giallozafferano.it/Cookies.html,12,"[['Farina 00', '195g'], ['Burro', '100g'], ['B...","Per preparare i cookies, assicuratevi che il b..."
2,2,Pancake allo sciroppo d'acero,Dolci,https://ricette.giallozafferano.it/Pancakes-al...,4,"[['Burro', '25g'], ['Farina 00', '125g'], ['Uo...",Iniziamo la preparazione dei pancake fondendo ...
3,3,Crema al mascarpone,Dolci,https://ricette.giallozafferano.it/Crema-al-ma...,4,"[['Mascarpone', '500g'], ['Zucchero', '125g'],...",Per preparare la crema al mascarpone versate i...
4,4,Crepe dolci e salate (ricetta base),Dolci,https://ricette.giallozafferano.it/Crepes-dolc...,15,"[['Uova', '3'], ['Farina 00', '250g'], ['Latte...",Per preparare le crepe dolci e salate iniziate...


In [39]:
print(df['Ingredienti'][0])

[['Mascarpone', '750g'], ['Uova', '260g'], ['Savoiardi', '250g'], ['Zucchero', '120g'], ['Caffè', '300g'], ['Cacao amaro in polvere', 'q.b.']]


In [40]:
row = df['Ingredienti'][0]
def getIngredient(row):
    row = row[2:-2]
    row = row.split('], [')
    print(row)
    res = []
    for item in row : 
        item = item.split(', ')
        tempres = []
        for item2 in item:
            tempres += [item2[1:-1]] 
        res += [tempres]
    return res
getIngredient(row)

["'Mascarpone', '750g'", "'Uova', '260g'", "'Savoiardi', '250g'", "'Zucchero', '120g'", "'Caffè', '300g'", "'Cacao amaro in polvere', 'q.b.'"]


[['Mascarpone', '750g'],
 ['Uova', '260g'],
 ['Savoiardi', '250g'],
 ['Zucchero', '120g'],
 ['Caffè', '300g'],
 ['Cacao amaro in polvere', 'q.b.']]

In [41]:
def getAllIng(df):
    allingredients = []
    for row in df['Ingredienti']:
        allingredients += [getIngredient(row)]
    return allingredients
ingredients = getAllIng(df)
print(len(ingredients))

["'Mascarpone', '750g'", "'Uova', '260g'", "'Savoiardi', '250g'", "'Zucchero', '120g'", "'Caffè', '300g'", "'Cacao amaro in polvere', 'q.b.'"]
["'Farina 00', '195g'", "'Burro', '100g'", "'Bicarbonato', '1pizzico'", "'Uova', '55g'", "'Zucchero di canna', '100g'", "'Zucchero', '100g'", "'Gocce di cioccolato fondente', '200g'", "'Sale fino', '1pizzico'"]
["'Burro', '25g'", "'Farina 00', '125g'", "'Uova', '2'", "'Latte intero', '200g'", "'Lievito in polvere per dolci', '6g'", "'Zucchero', '15g'", "'Sciroppo di acero', 'q.b.'"]
["'Mascarpone', '500g'", "'Zucchero', '125g'", "'Tuorli', '4'", "'Acqua', '50g'", "'Cacao amaro in polvere', 'q.b.'"]
["'Uova', '3'", "'Farina 00', '250g'", "'Latte intero', '500ml'", "'Burro', 'q.b.'"]
["'Semola di grano duro rimacinata', '350g'", "'Farina 00', '150g'", "'Spinaci', '250g'", "'Uova', '2'", "'Tuorli', '3'", "'Carne bovina', '300g'", "'Pancetta', '150g'", "'Carote', '50g'", "'Sedano', '50g'", "'Cipolle dorate', '50g'", "'Vino rosso', '100g'", "'Passata

In [53]:
def getAllUniqueIngredients(ing):
    res = []
    for recipe in ing : 
        for ingredient in recipe:
            res += [ingredient[0]]

    uniques = np.unique(res)
    print(uniques)
    print(len(uniques))
    print(len(ing))
    return uniques

getAllUniqueIngredients(ingredients)

['' 'Abbacchio' 'Acai in polvere' ... 'Zucchine trombetta' 'codette'
 'zenzero candito']
1642
5939


array(['', 'Abbacchio', 'Acai in polvere', ..., 'Zucchine trombetta',
       'codette', 'zenzero candito'], dtype='<U56')

In [ ]:
print("nbRecipes : ")